In [107]:
import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine, text
import os

In [135]:
connection_string = 'sqlite:///db/neat_stock_data.db'

In [136]:
def download_stock_data(ticker: str):
    data = yf.download(ticker, start='2020-01-01')
    return data

def upload_stock_data_to_sql(data:pd.DataFrame, ticker: str):
    df = pd.DataFrame(data)
    df['Ticker'] = ticker
    df.drop(columns=['High','Low','Close'],inplace=True)
    engine = create_engine(connection_string, echo=True)
    df.to_sql('stock_data', con=engine, if_exists='replace', index=False)

In [137]:
ticker = 'BTC-USD'
data = download_stock_data(ticker)
upload_stock_data_to_sql(data,ticker)

[*********************100%%**********************]  1 of 1 completed


OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [125]:
def query_data(ticker: str):
    engine = create_engine(connection_string)
    table_name = 'stock_data'
    query = f"SELECT * FROM {table_name} WHERE Ticker = '{ticker}'"
    
    return pd.read_sql(query, engine)

def query_all():
    engine = create_engine(connection_string)
    table_name = 'stock_data'
    query = f"SELECT * FROM {table_name}"

    return pd.read_sql(query, engine)

def delete_data(ticker: str):
    engine = create_engine(connection_string)
    table_name = "stock_data"
    condition = f"Ticker = '{ticker}'"

    with engine.connect() as connection:
        result = connection.execute(text(f"DELETE FROM {table_name} WHERE {condition} "))
        connection.commit()
    
        print(f"Deleted {result.rowcount} row(s) from {table_name}")

    connection.close

In [92]:
delete_data('BTC-USD')

Deleted 1561 row(s) from stock_data


In [131]:
df = query_all()
df

,Open,Adj Close,Volume,Ticker
0,7194.892090,7200.174316,18565664997,BTC-USD
1,7202.551270,6985.470215,20802083465,BTC-USD
2,6984.428711,7344.884277,28111481032,BTC-USD
3,7345.375488,7410.656738,18444271275,BTC-USD
4,7410.451660,7411.317383,19725074095,BTC-USD
...,...,...,...,...
1556,68515.757812,67837.640625,33748230056,BTC-USD
1557,67840.570312,68896.109375,19967785809,BTC-USD
1558,68897.109375,69362.554688,21204930369,BTC-USD
1559,69362.554688,71631.359375,37261432669,BTC-USD


In [108]:
def delete_db(db_file):
    try:
        os.remove(db_file)
        print(f"SQLite databse '{db_file}' deleted successfully")
    
    except FileNotFoundError:
        print(f"Could not find SQLite database '{db_file}'")

    except Exception as e:
        print(f"An error occured: {e}")    

In [109]:
delete_db("sqlite:///stock_data.db")

Could not find SQLite database 'sqlite:///stock_data.db'
